In [71]:
import hashlib
import datetime
import json
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [72]:
class Transakcija:
    def __init__(self, sender_wallet, receiver_wallet, amount):
        self.sender = sender_wallet.address
        self.receiver = receiver_wallet.address
        self.amount = amount

    def to_dict(self):
        return {
            "sender": self.sender,
            "receiver": self.receiver,
            "amount": self.amount
        }

    def __repr__(self):
        return f"Transakcija({self.sender} -> {self.receiver}, {self.amount})"

In [73]:
class Wallet:
    def __init__(self, ime):
        self.ime=ime
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        self.public_key = self.private_key.public_key()
        self.address = hashlib.sha256(
            self.public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        ).hexdigest()

    def get_public_key_pem(self):
        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')

    def __repr__(self):
        return f"Wallet({self.ime})"

In [74]:
class Blok:
    def __init__(self, index, transakcije, previous_hash=""):
        self.index = index
        self.transakcije = transakcije
        self.timestamp = datetime.datetime.now()
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "transakcije": [t.to_dict() for t in self.transakcije],
            "timestamp": str(self.timestamp),
            "previous_hash": self.previous_hash
        }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()

    def __repr__(self):
        return json.dumps({
            "index": self.index,
            "timestamp": str(self.timestamp),
            "transakcije": [t.to_dict() for t in self.transakcije],
            "previous_hash": self.previous_hash,
            "hash": self.hash
        }, indent=4)

In [75]:
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        genesis_wallet = Wallet("GENESIS")
        network_wallet = Wallet("network")
        return Blok(0, [Transakcija(genesis_wallet, network_wallet, 0)], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def push(self, novi_blok):
        novi_blok.previous_hash = self.get_latest_block().hash
        novi_blok.hash = novi_blok.calculate_hash()
        self.chain.append(novi_blok)

    def is_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            if current.hash != current.calculate_hash():
                return False
            if current.previous_hash != previous.hash:
                return False
        return True

    def __repr__(self):
        return "\n".join([repr(blok) for blok in self.chain])

In [76]:
profesor=Wallet("profesor")
student=Wallet("student")
cryptowhale=Wallet("cryptowhale")

In [77]:
t1=Transakcija(profesor,student,5)
t2=Transakcija(cryptowhale,student,5000)
t3=Transakcija(cryptowhale,profesor,200)
t4=Transakcija(student,profesor,2000)
t5=Transakcija(profesor,cryptowhale,5)

In [78]:
blockchain=Blockchain()

In [ ]:
Blok1=Blok(1,[t1])
Blok2=Blok(2,[t2])
Blok3=Blok(3,[t3,t4,t5])

In [85]:
blockchain.push(Blok1)
blockchain.push(Blok2)
blockchain.push(Blok3)

In [86]:
print(blockchain)

{
    "index": 0,
    "timestamp": "2025-11-10 18:52:29.342854",
    "transakcije": [
        {
            "sender": "e44ca995fd65e994b174e80d1f5c769149f802d107b678f142e391b95fcf7078",
            "receiver": "555c634cfce41673d1192cf945e66c5091be22ee968aa5c71d15a19efa167299",
            "amount": 0
        }
    ],
    "previous_hash": "0",
    "hash": "ed9fa5ed18cafd0846b0f6db7b9cfa7e21cbeee31728ac1bde61f01809783d08"
}
{
    "index": 1,
    "timestamp": "2025-11-10 18:53:27.636881",
    "transakcije": [
        {
            "sender": "cd865538f9aa524f4cd771525d6fe09e86fa2b23ab6af60f1e0d4085a1a10638",
            "receiver": "2a7f64dad205009fe712e0c0a304f0dc10655193ec8911eec60a027e37e2dc68",
            "amount": 5
        }
    ],
    "previous_hash": "ed9fa5ed18cafd0846b0f6db7b9cfa7e21cbeee31728ac1bde61f01809783d08",
    "hash": "f2058567f000ff2c643c6e82b0da207f3c3cef1d489a85ccb1889a0924f17d52"
}
{
    "index": 2,
    "timestamp": "2025-11-10 18:53:27.636939",
    "transakcije": [